[Reference](https://medium.com/atoti/complementing-rdbms-with-multidimensional-analysis-3e7b45524399)

In [1]:
pip install atoti[jupyterlab,sql]

     |████████████████████████████████| 146.6 MB 33 kB/s 
     |████████████████████████████████| 25.6 MB 1.5 MB/s 
     |████████████████████████████████| 35.7 MB 182 kB/s 
     |████████████████████████████████| 198 kB 49.4 MB/s 
     |████████████████████████████████| 6.0 MB 14.6 MB/s 
     |████████████████████████████████| 23.6 MB 1.4 MB/s 
     |████████████████████████████████| 8.6 MB 53.5 MB/s 
     |████████████████████████████████| 428 kB 23.8 MB/s 
     |████████████████████████████████| 396 kB 58.8 MB/s 
     |████████████████████████████████| 61 kB 7.7 MB/s 
     |████████████████████████████████| 79 kB 7.7 MB/s 
     |████████████████████████████████| 129 kB 56.9 MB/s 
     |████████████████████████████████| 53 kB 1.9 MB/s 
     |████████████████████████████████| 71 kB 235 kB/s 
  Attempting uninstall: tornado
    Found existing installation: tornado 5.1.1
    Uninstalling tornado-5.1.1:
      Successfully uninstalled tornado-5.1.1
  Attempting uninstall: jsonschema
    F

In [3]:
import atoti as tt
session = tt.create_session()

Welcome to atoti 0.6.3!

    By using this community edition, you agree with the license available at https://docs.atoti.io/latest/eula.html.
    Browse the official documentation at https://docs.atoti.io.
    Join the community at https://www.atoti.io/register.

    atoti collects telemetry data, which is used to help understand how to improve the product.
    If you don't wish to send usage data, set the ATOTI_DISABLE_TELEMETRY environment variable to True.

    You can hide this message by setting the ATOTI_HIDE_EULA_MESSAGE environment variable to True.


In [5]:
# session = tt.create_session(
#     config={
#         "extra_jars": ["./driver/vertica-jdbc-11.0.0-0.jar"],
#     }
# )

In [7]:
sales_table = session.read_sql(
    "SELECT * FROM [atoti].[atoti].[sales];",
    url="jdbc:sqlserver://localhost:1433;databaseName=atoti;user=atotiUser;password=atotiPass",
    table_name="sales",
    keys=["Sale ID", "Date"],
)

In [8]:
sales_table.head()

In [9]:
policy_tbl = session.read_sql(
    """SELECT CUST_ID , PP.POLICY, POLICY_STATUS, 'Property' AS POLICY_TYPE,
    DATE(QUOTE_DATE) AS QUOTE_DATE, DATE(COVER_START) AS COVER_START,
    DATE(LAPSED_DATE) AS LAPSED_DATE, DATE(CANCELLED_DATE) AS CANCELLED_DATE, DATE(MATURITY_DATE) AS MATURITY_DATE,
   PAYMENT_METHOD,PAYMENT_FREQUENCY,SUM_INSURED,OWNERSHIP,PLAN_TYPE,BASIC_PREMIUM, NUM_SUBPOLICY, 'N' as IS_SUB  
    FROM POLICY_PROPERTY PP
    LEFT JOIN (SELECT POLICY, COUNT(DISTINCT(SUB_POLICY)) AS NUM_SUBPOLICY FROM ADDITIONAL_COVERAGE GROUP BY POLICY) ADDONS ON ADDONS.POLICY = PP.POLICY;""",
    url="jdbc:vertica://192.168.56.101:5433/VMart?user=dbadmin&password=password",
    table_name="policy",
    keys=["CUST_ID", "POLICY"],
    driver="com.vertica.jdbc.Driver",
)

In [10]:
policy_tbl.load_sql(
    """select CUST_ID ,POLICY, POLICY_STATUS, 'Vehicle' as POLICY_TYPE,
    DATE(QUOTE_DATE) as QUOTE_DATE, DATE(COVER_START) as COVER_START,
    DATE(LAPSED_DATE) as LAPSED_DATE, DATE(CANCELLED_DATE) as CANCELLED_DATE, DATE(MATURITY_DATE) as MATURITY_DATE,
    PAYMENT_METHOD,PAYMENT_FREQUENCY,SUM_INSURED,OWNERSHIP,PLAN_TYPE,BASIC_PREMIUM, 0 as NUM_SUBPOLICY, 'N' as IS_SUB   
    from policy_vehicle;""",
    url="jdbc:vertica://192.168.56.101:5433/VMart?user=dbadmin&password=password",
    driver="com.vertica.jdbc.Driver",
)

# Creating multiple tables


In [11]:
coverage_tbl = session.read_sql(
    "select * from coverage;",
    url="jdbc:vertica://192.168.56.101:5433/VMart?user=dbadmin&password=password",
    table_name="coverage",
    keys=["POLICY", "COVERAGE_TYPE"],
    driver="com.vertica.jdbc.Driver",
)

policy_tbl.join(coverage_tbl)

# Creating multidimensional cube


In [12]:
cube = session.create_cube(policy_tbl, "Insurance")
cube.schema

# Hierarchies and measures creation


In [13]:
m["Claim Amount"] = tt.agg.sum(
    tt.value(claim_tbl["CLAIM_AMOUNT"]), scope=tt.scope.origin(l["POLICY"])
)